In [7]:
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Dataset
from azureml.core import Workspace,ScriptRunConfig,Experiment, Run

**Dataset**

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
ws = Workspace.from_config()
experiment_name = 'lv-hyperparameter'

exp=Experiment(ws, experiment_name)

In [9]:
run = exp.start_logging()

In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


   # Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


**Hyperdrive Configuration**

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.dnn import TensorFlow
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
import os,shutil


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.15, evaluation_interval=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        '--C': choice(0,0.25,0.5,1),
        '--max_iter': choice(500,1000,5000,10000)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
# Reference: lesson 6.3: copying the training file into the script folder
shutil.copy('./train.py', script_folder)
    
script_params={
    '--datastore-dir': ws.get_default_datastore().as_mount(),
}

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='training', 
                     script_params=script_params,
                    compute_target=cpu_cluster,
                    entry_script='train.py',
                    pip_packages=['joblib']
                   )


hyperdrive_run_config = HyperDriveConfig(estimator = estimator, 
                                            hyperparameter_sampling = param_sampling, 
                                            policy = early_termination_policy,
                                            primary_metric_name = "Accuracy",
                                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs = 20,
                                            max_concurrent_runs = 4)

**Run Details**

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the RunDetails widget to show the different experiments.

In [12]:
#TODO: Submit your experiment
hd_run = exp.submit(hyperdrive_run_config)
RunDetails(Run(exp, hd_run.id)).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

**Best Model**

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
import joblib
# Get your best run and save the model from that run.

best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])

Best Run Id:  HD_78e1907f-4c03-4212-a8dd-6ac463add8d6_0

 Accuracy: 0.9126984126984127

 learning rate: 0.5

 keep probability: 10000
